# Reconnaissance d'entités nommées avec SpaCy

La documentation est accessible ici: https://spacy.io/api

## Imports

In [1]:
from collections import defaultdict
import spacy
from spacy.lang.fr.examples import sentences
!python -m spacy download fr_core_news_md

2025-10-21 20:16:49.896040: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-10-21 20:16:51.424457: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
DEPRECATION: https://github.com/explosion/spacy-models/releases/download/-fr_core_news_md/-fr_core_news_md.tar.gz#egg===fr_core_news_md contains an egg fragment with a non-PEP 508 name. pip 25.3 will enforce this behaviour change. A possible replacement is to use the req @ url syntax, and remove the egg fragment. Discussion can be found at https://github.com/pypa/pip/issues/13157
ERROR: Invalid requirement: '

In [4]:
nlp = spacy.load('fr_core_news_md')

## Exemple sur un corpus de test fourni par SpaCy

In [5]:
# Imprimer le corpus de Spacy
sentences

['Apple cherche à acheter une start-up anglaise pour 1 milliard de dollars',
 "Les voitures autonomes déplacent la responsabilité de l'assurance vers les constructeurs",
 "San Francisco envisage d'interdire les robots coursiers sur les trottoirs",
 'Londres est une grande ville du Royaume-Uni',
 'L’Italie choisit ArcelorMittal pour reprendre la plus grande aciérie d’Europe',
 "Apple lance HomePod parce qu'il se sent menacé par l'Echo d'Amazon",
 "La France ne devrait pas manquer d'électricité cet été, même en cas de canicule",
 'Nouvelles attaques de Trump contre le maire de Londres',
 'Où es-tu ?',
 'Qui est le président de la France ?',
 'Où est la capitale des États-Unis ?',
 'Quand est né Barack Obama ?']

In [6]:
# Isoler la première phrase
sent = sentences[0]
sent

'Apple cherche à acheter une start-up anglaise pour 1 milliard de dollars'

In [7]:
# Traiter la phrase avec Spacy
doc = nlp(sent)

In [8]:
type(doc)

spacy.tokens.doc.Doc

In [9]:
doc.text

'Apple cherche à acheter une start-up anglaise pour 1 milliard de dollars'

In [10]:
doc.to_json()

{'text': 'Apple cherche à acheter une start-up anglaise pour 1 milliard de dollars',
 'ents': [{'start': 0, 'end': 5, 'label': 'ORG'}],
 'sents': [{'start': 0, 'end': 72}],
 'tokens': [{'id': 0,
   'start': 0,
   'end': 5,
   'tag': 'PROPN',
   'pos': 'PROPN',
   'morph': '',
   'lemma': 'Apple',
   'dep': 'nsubj',
   'head': 1},
  {'id': 1,
   'start': 6,
   'end': 13,
   'tag': 'VERB',
   'pos': 'VERB',
   'morph': 'Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin',
   'lemma': 'cherche',
   'dep': 'ROOT',
   'head': 1},
  {'id': 2,
   'start': 14,
   'end': 15,
   'tag': 'ADP',
   'pos': 'ADP',
   'morph': '',
   'lemma': 'à',
   'dep': 'mark',
   'head': 3},
  {'id': 3,
   'start': 16,
   'end': 23,
   'tag': 'VERB',
   'pos': 'VERB',
   'morph': 'VerbForm=Inf',
   'lemma': 'acheter',
   'dep': 'xcomp',
   'head': 1},
  {'id': 4,
   'start': 24,
   'end': 27,
   'tag': 'DET',
   'pos': 'DET',
   'morph': 'Definite=Ind|Gender=Fem|Number=Sing|PronType=Art',
   'lemma': 'un',
   

In [11]:
# Appliquer le test sur toutes les phrases
for sent in sentences:
    doc = nlp(sent)
    entities = []
    for ent in doc.ents:
        entities.append(f"{ent.text} ({ent.label_})")
    if entities:
        print(f"'{doc.text}' contient les entités suivantes : {', '.join(entities)}")
    else:
        print(f"'{doc.text}' ne contient aucune entité")

'Apple cherche à acheter une start-up anglaise pour 1 milliard de dollars' contient les entités suivantes : Apple (ORG)
'Les voitures autonomes déplacent la responsabilité de l'assurance vers les constructeurs' ne contient aucune entité
'San Francisco envisage d'interdire les robots coursiers sur les trottoirs' contient les entités suivantes : San Francisco (LOC)
'Londres est une grande ville du Royaume-Uni' contient les entités suivantes : Londres (LOC), Royaume-Uni (LOC)
'L’Italie choisit ArcelorMittal pour reprendre la plus grande aciérie d’Europe' contient les entités suivantes : L’Italie (LOC), ArcelorMittal (ORG), Europe (LOC)
'Apple lance HomePod parce qu'il se sent menacé par l'Echo d'Amazon' contient les entités suivantes : Apple (ORG), HomePod (MISC), Echo (MISC), Amazon (ORG)
'La France ne devrait pas manquer d'électricité cet été, même en cas de canicule' contient les entités suivantes : La France (LOC)
'Nouvelles attaques de Trump contre le maire de Londres' contient les e

## Appliquer la reconnaissance d'entités nommées sur notre corpus

In [12]:
# Charger le texte
n=1000000
text = open("../data/all.txt", encoding='utf-8').read()[:n]

In [13]:
%%time
# Traiter le texte

doc = nlp(text)

CPU times: total: 23.6 s
Wall time: 24.2 s


In [15]:
# Compter les entités
people = defaultdict(int)
for ent in doc.ents:
    if ent.label_ == "PER" and len(ent.text) > 3:
        people[ent.text] += 1

In [16]:
# Trier et imprimer

sorted_people = sorted(people.items(), key=lambda kv: kv[1], reverse=True)

for person, freq in sorted_people[:50]:
    print(f"{person} apparait {freq} fois dans le corpus")

Thérèse apparait 24 fois dans le corpus
Guilbert apparait 22 fois dans le corpus
M. Wilson apparait 20 fois dans le corpus
Rossel apparait 20 fois dans le corpus
Gioconda apparait 18 fois dans le corpus
Molière apparait 17 fois dans le corpus
Enguerrand apparait 17 fois dans le corpus
Wilson apparait 16 fois dans le corpus
Daubrac apparait 15 fois dans le corpus
Parentis apparait 13 fois dans le corpus
Savinien apparait 13 fois dans le corpus
Octave apparait 12 fois dans le corpus
Agence Rossel apparait 12 fois dans le corpus
Monsieur apparait 12 fois dans le corpus
Edwige apparait 12 fois dans le corpus
duc d’Aumale apparait 11 fois dans le corpus
jo no apparait 9 fois dans le corpus
Jules Ferry apparait 9 fois dans le corpus
Naissances apparait 9 fois dans le corpus
Reine apparait 9 fois dans le corpus
Monseigneur apparait 9 fois dans le corpus
Graux apparait 8 fois dans le corpus
Finet apparait 8 fois dans le corpus
Ali-Baba apparait 8 fois dans le corpus
Henri apparait 8 fois dans 

Exercice: essayez de lister les lieux (LOC) et les organisations (ORG) les plus mentionnées dans le corpus